In [ ]:
%pip install -e .
from pathlib import Path
import matplotlib.pyplot as plt
from analysis_pipeline.tdms_tools import TDMSDataset  # <- from your module

In [ ]:
FOLDER = r"C:\your_path_here"   # folder containing .tdms files
CHANNEL = "wfg"             # channel to plot
GROUP   = None              # or a specific TDMS group

# Time window relative to dataset start [seconds]
T0_REL_S = 0.999             
T1_REL_S = 1.001            

# Stream size (bigger -> fewer I/O calls)
CHUNK_SAMPLES = 2_000_000

# Optional: force a dtype during read (None = native TDMS dtype)
FORCE_DTYPE = None

In [ ]:
# Define the dataset with respect to your folder
ds = TDMSDataset(FOLDER, group=GROUP)

# Returns:
#  - t_rel_s: float seconds since dataset start (numpy 1D array)
#  - y: samples (same length as t_rel_s)
#  - ds_start_abs: absolute start time of the dataset (datetime64[ns])
t_rel_s, y, ds_start_abs = ds.read_window_rel(
    CHANNEL,
    T0_REL_S,
    T1_REL_S,
    chunk_samples=CHUNK_SAMPLES,
    dtype=FORCE_DTYPE,
)

#Check load in
if t_rel_s.size == 0:
    print("No samples found in the requested relative window.")
else:
    print(f"Loaded {y.size} samples in "
          f"[{T0_REL_S:.3f}s, {T1_REL_S:.3f}s] since dataset start "
          f"({str(ds_start_abs).replace('T',' ')})")


In [ ]:
# Example Plot
if t_rel_s.size:
    plt.figure(figsize=(10, 4))
    plt.plot(t_rel_s, y, linewidth=1)
    plt.xlabel("Time since dataset start [s]")
    plt.ylabel(CHANNEL)
    title_time = str(ds_start_abs).replace("T", " ")
    plt.title(f"{CHANNEL} — {Path(FOLDER).name}\nDataset start: {title_time}\n"
              f"Window: {T0_REL_S:.3f}s → {T1_REL_S:.3f}s")
    plt.tight_layout()
    plt.show()
